# Smoothing the curve 📈

## From dictionaries to tables

Let's recap the concept of dictionary and how can we generate one:

a| 1
b| 2


When a dictionary contains lists of equal length, and the keys are symbols, we create a *column dictionary*.

We can convert a column dictionary into a table by applying `flip`. This turns the dictionary into a table where keys become column names and values become the rows.

<div class="alert alert-warning">
One of the keys of why kdb is so efficient is that kdb database is column oriented rather than row-oriented, as in traditional relational databases like SQL. This means data is stored by columns, not rows.
</div>

<div class="alert alert-success">
  In kdb+, tables are first-class citizens, meaning they reside in memory just like lists and other data structures. They form the core of kdb+, enabling efficient storage and querying of large datasets, especially in time-series analysis.
</div>

## Generating items

Going back to our problem, we need to gen a price tabl that look like this:

<img src="../pics/priceTable.JPG" alt="Italian Trulli">

What do we need to gen one example of our price table?
- Temporal column
- Asset column
- Price column



To build the temporal column we have the built-in `minute` type, that simplifies working with time values, allowing you to perform operations directly on hours.

If you look closely you can realise that we need something more, we need to sort ascendelly the hours, so we are going to use `asc` keyword to handle that. And, then we store our list in `ts`

<div class="alert alert-warning">
Look a the `s#` at the beginning of the output. It means that, that list has the <strong>sorted</strong> atribbute, meaning the data is recognized by q as being in sorted order, which allows for faster lookups and optimizations during queries
</div>

Now, to generate our list of assets, we are going to use *symbols*. Symbols are atomic data types rather than lists of characters, unlike strings. This makes them more efficient for use in tables, as symbols store references to their unique value rather than the full string.

So similarly, we can create a list of symbols for the assets and store it in `syms`

Finally, we are going to need to generate random data, lets use `roll` `?`, for example to roll a dice 10 times

Left number is the number of times and right number is the limit number, so let's first generate starting prices 

And then we generate randomly the price jumps between each instant of time. In order to sum the jumps to the starting points, we need to reshape the starting points, so both lists conform

As last point, random data is generated based on a seed value, which controls the sequence of random numbers. By fixing the seed using `\S`, we ensure that the same random output is reproducible across different runs.

Now, we have the ingredients (variables `ts`, `syms` and `p`) to create our column dictionary ...

... and our table will be ready just flipping the dictionary

Finally, there's another syntax to create a table

### SQL in the Q world

By now, we know what a table is and how to create one in q. Next, we'll explore what we can do with tables using qSQL, a set of SQL-like functions in q. These operations will feel familiar if you have SQL experience.

<div class="alert alert-info">
Using your previous SQL knowledge, how would you select all columns from a table?
<div>

Filtering rows is just as simple.

We can also create new columns or select specific ones. Here's an example of calculating the average price for `AAPL`

In qSQL, `update` behaves like `select` but it creates or modifies columns preserving the original table. For example, let's add a `mean` column to our table.

t     sym  price    mean    
----------------------------
09:00 AAPL 96.26213 83.36099
09:00 AMZN 78.44964 83.36099
09:00 GOOG 75.3932  83.36099
09:01 AAPL 96.27035 83.36099
09:01 AMZN 78.42006 83.36099
09:01 GOOG 75.3768  83.36099
09:02 AAPL 96.27845 83.36099
09:02 AMZN 78.42342 83.36099
09:02 GOOG 75.37487 83.36099


Grouping and aggregating is also straightforward using the `by` clause, similar to SQL's `GROUP BY`.

Hmm, what's that vertical line we see in our grouped table?

This is a `keyed table`. It's actually a dictionary that maps key records to value records. We can access individual records using dictionary-like syntax.

But if we want to do lookup on multiple elements, we can't ...

... because keys are tables just as values are, so we'll use a table of our own to index

## Getting previous element

Focusing back in the problem, we aim to compute the average of the current price with the previous price for each symbol. To access the previous value in a sequence, we can use the `prev` keyword in q. Let's see how it works.

Notice that the first element is `0N` (null) since there's no preceding value. To avoid this, we can use the `fill` (`^`) keyword to fill the nulls with appropriate values

Let's now integrate this logic with our table operations using qSQL syntax to compute the average for each symbol's prices.

Finally, we just need to encapsulate it in a function

We got it!

## InterConnection

kdb+ supplies interprocess communication (IPC), which enables communication between different q processes. To demonstrate this, we need to start the server side of a q process.

We can begin by listening for incoming requests by using `\p` followed by a port number.